In [1]:
import numpy as np
import pandas as pd
from time import time
import re
import random
import pickle
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('D:/Capstone/dataset/emails_total_preprocessed_raw.csv')


In [3]:
df

,Unnamed: 0,message,user,folder,from,type,subject,date,bcc,email,body,word_count,email_counts
0,0,Message-ID: <18782981.1075855378110.JavaMail.e...,allen-p,_sent_mail/1,phillip.allen@enron.com,sent,no subject,"Mon, 14 May 2001 16:39:00",None,phillip.allen@enron.com,Here is our forecast,6,2082
1,1,Message-ID: <15464986.1075855378456.JavaMail.e...,allen-p,_sent_mail/10,phillip.allen@enron.com,sent,no subject,"Fri, 4 May 2001 13:51:00",None,phillip.allen@enron.com,Traveling to have a business meeting takes th...,141,2082
2,2,Message-ID: <24216240.1075855687451.JavaMail.e...,allen-p,_sent_mail/100,phillip.allen@enron.com,sent,test,"Wed, 18 Oct 2000 03:00:00",None,phillip.allen@enron.com,test successful way to go,7,2082
3,3,Message-ID: <13505866.1075863688222.JavaMail.e...,allen-p,_sent_mail/1000,phillip.allen@enron.com,sent,no subject,"Mon, 23 Oct 2000 06:13:00",None,phillip.allen@enron.com,Randy Can you send me a schedule of the salar...,35,2082
4,4,Message-ID: <30922949.1075863688243.JavaMail.e...,allen-p,_sent_mail/1001,phillip.allen@enron.com,sent,Hello,"Thu, 31 Aug 2000 05:07:00",None,phillip.allen@enron.com,Let's shoot for Tuesday at,8,2082
5,5,Message-ID: <30965995.1075863688265.JavaMail.e...,allen-p,_sent_mail/1002,phillip.allen@enron.com,sent,Hello,"Thu, 31 Aug 2000 04:17:00",None,phillip.allen@enron.com,Greg How about either next Tuesday or Thursda...,10,2082
6,6,Message-ID: <16254169.1075863688286.JavaMail.e...,allen-p,_sent_mail/1003,phillip.allen@enron.com,sent,no subject,"Tue, 22 Aug 2000 07:44:00",None,phillip.allen@enron.com,Please cc the following distribution list wit...,29,2082
7,7,Message-ID: <17189699.1075863688308.JavaMail.e...,allen-p,_sent_mail/1004,phillip.allen@enron.com,sent,PRC review phone calls,"Fri, 14 Jul 2000 06:59:00",None,phillip.allen@enron.com,any morning between and,6,2082
8,8,Message-ID: <20641191.1075855687472.JavaMail.e...,allen-p,_sent_mail/101,phillip.allen@enron.com,sent,High Speed Internet Access,"Tue, 17 Oct 2000 02:26:00",None,phillip.allen@enron.com,login pallen pw kedavis I don't think these ...,31,2082
9,9,Message-ID: <30795301.1075855687494.JavaMail.e...,allen-p,_sent_mail/102,phillip.allen@enron.com,sent,fixed forward or other Collar floor gas price...,"Mon, 16 Oct 2000 06:44:00",None,phillip.allen@enron.com,fixed forward or other Collar floor gas price...,266,2082


In [4]:
df['domain']=df['from'].str.extract(r"@(.*)")

In [5]:
def email_clean(domain, email):
    output=[]
    for i in range(len(domain)):
        if domain[i] =='enron.com':
            output.append(email[i])
        else:
            output.append(domain[i])
    return output

In [6]:
df['from_clean'] = email_clean(df['domain'],df['from'])

In [7]:
user_list=df['user'].unique().tolist()
email_list=df['from_clean'].unique().tolist()

In [8]:
df_employee = pd.DataFrame({'uuid':[i for i in range(len(user_list))],'user':user_list})

df_email = pd.DataFrame({'eid':[i for i in range(len(email_list))],'from_clean':email_list})


df_messages = df[['Unnamed: 0','from_clean','user','type','subject','body','date', 'bcc']]
df_messages.columns = ['mid','from_clean','user','type','subject','body','date', 'bcc']

#email_counts=df['from'].value_counts().rename('email_counts')
#df_email=df.merge(email_counts.to_frame(),
#             left_on='from',
#             right_index=True)

In [9]:
df_messages.shape

(473749, 8)

In [10]:
def make_df(df,cnt_what,by2_criteria,by2,by1,by_list):
    new_df= pd.DataFrame({})
    for i in by_list:
        a=df[(df[by1]==i) & (df[by2]==by2_criteria)][cnt_what].value_counts()
        b=a.to_frame()
        b['email'] =b.index
        b['user']=i
        new_df= pd.concat([new_df,b])
    return new_df

In [11]:
df_alias=make_df(df,'from_clean','sent','type','user',user_list)
df_alias.reset_index(drop=True, inplace=True)
df_alias.columns=['sent_no','email','user']

In [12]:
#verify if we should keep aliases that have less than 10 emails sent. (did they save received emails to 'sent' folder)
check=df_alias[df_alias.sent_no<10]
len(check)
#check.reset_index(drop=True, inplace=True)

130

In [13]:
#check.loc[:80]

In [14]:
#alias 'vkaminski@aol.com' would be lost after domain appending. 
#keep
keep_list=['l..gay@enron.com','michele.lokay@enron.com',
           'stacey.white@enron.com','paul.thomas@enron.com',
           'eric.linder@enron.com','steven.merris@enron.com']

In [15]:
df_alias.head()

,sent_no,email,user
0,1141,phillip.allen@enron.com,allen-p
1,290,k..allen@enron.com,allen-p
2,8,ina.rangel@enron.com,allen-p
3,1,pam.butler@enron.com,allen-p
4,5017,jeff.dasovich@enron.com,dasovich-j


In [16]:
df1 = df_alias[df_alias['sent_no']>9]
for i in keep_list: 
    df1 = pd.concat([df1, df_alias[df_alias['email']==i]])

df_alias=df1

check=df_alias[df_alias.sent_no<10]
df_alias.reset_index(drop=True, inplace=True)

In [17]:
display(df_alias.head())
display(df_alias.tail())

,sent_no,email,user
0,1141,phillip.allen@enron.com,allen-p
1,290,k..allen@enron.com,allen-p
2,5017,jeff.dasovich@enron.com,dasovich-j
3,28,frank.ermis@enron.com,ermis-f
4,827,mike.grigsby@enron.com,grigsby-m


,sent_no,email,user
209,1,michele.lokay@enron.com,lokay-m
210,1,stacey.white@enron.com,white-s
211,8,paul.thomas@enron.com,thomas-p
212,6,eric.linder@enron.com,linder-e
213,3,steven.merris@enron.com,merriss-s


In [18]:
for i in keep_list:
    display(df_alias[df_alias.email == i])

,sent_no,email,user
208,8,l..gay@enron.com,gay-r


,sent_no,email,user
209,1,michele.lokay@enron.com,lokay-m


,sent_no,email,user
210,1,stacey.white@enron.com,white-s


,sent_no,email,user
211,8,paul.thomas@enron.com,thomas-p


,sent_no,email,user
212,6,eric.linder@enron.com,linder-e


,sent_no,email,user
213,3,steven.merris@enron.com,merriss-s


In [19]:
#df_email = df_email.merge(df_employee, 
#                          on ='user')
#df_email.drop(columns='user',inplace=True)
#df_messages.columns = ['eid','email','type','subject','body','date', 'bcc']

In [20]:
print('number of aliases:', df_alias.shape[0])
df_alias=df_alias.drop(columns ='sent_no')
df_alias.columns =['from_clean','user']

number of aliases: 214


In [21]:
df_alias = df_alias.append(pd.Series(['sharris1@enron.com','harris-s'], index=df_alias.columns ), ignore_index=True)
df_alias = df_alias.append(pd.Series(['steven.harris@enron.com','harris-s'], index=df_alias.columns ), ignore_index=True)

In [22]:
keep_list

['l..gay@enron.com',
 'michele.lokay@enron.com',
 'stacey.white@enron.com',
 'paul.thomas@enron.com',
 'eric.linder@enron.com',
 'steven.merris@enron.com']

In [23]:
df_alias.head()

,from_clean,user
0,phillip.allen@enron.com,allen-p
1,k..allen@enron.com,allen-p
2,jeff.dasovich@enron.com,dasovich-j
3,frank.ermis@enron.com,ermis-f
4,mike.grigsby@enron.com,grigsby-m


In [24]:
len(df_messages)

473749

In [25]:
df_alias = df_alias.merge(df_email, on='from_clean')
df_alias = df_alias.merge(df_employee,on ='user')

In [26]:
df_alias[df_alias.duplicated('eid')]

,from_clean,user,eid,uuid
17,twanda.sweet@enron.com,cash-m,1831,113
128,greg.whalley@enron.com,whalley-l,103,78
129,liz.taylor@enron.com,whalley-l,234,78


In [27]:
df_alias[df_alias['eid']==1831]

,from_clean,user,eid,uuid
16,twanda.sweet@enron.com,sanders-r,1831,8
17,twanda.sweet@enron.com,cash-m,1831,113


In [28]:
df_alias[df_alias['eid']==103]

,from_clean,user,eid,uuid
126,greg.whalley@enron.com,whalley-g,103,77
128,greg.whalley@enron.com,whalley-l,103,78


In [29]:
df_alias[df_alias['eid']==234]

,from_clean,user,eid,uuid
127,liz.taylor@enron.com,whalley-g,234,77
129,liz.taylor@enron.com,whalley-l,234,78


In [30]:
df_email.to_csv('D:/Capstone/dataset/database/df_email.csv')

In [31]:
df_alias.to_csv('D:/Capstone/dataset/database/alias.csv')

In [32]:
df_messages=df_messages.merge(df_employee, on='user')
df_messages=df_messages.merge(df_email, on='from_clean')

In [33]:
df_messages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 473749 entries, 0 to 473748
Data columns (total 10 columns):
mid           473749 non-null int64
from_clean    473749 non-null object
user          473749 non-null object
type          473749 non-null object
subject       473749 non-null object
body          473749 non-null object
date          473749 non-null object
bcc           473749 non-null object
uuid          473749 non-null int64
eid           473749 non-null int64
dtypes: int64(3), object(7)
memory usage: 39.8+ MB


In [34]:
len(df_messages)

473749

In [35]:
#fix emails that are not captured in sent folders.
for i in range(len(df)):
    user = df_messages['uuid'][i]
    user_alias = df_alias[df_alias['uuid']==user]['eid'].tolist()
    if (df_messages['eid'][i] in user_alias) & (df_messages['type'][i] == 'received'):
        df_messages['type'][i] = 'sent'

C:\Anaconda3\envs\N4J\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [36]:
df_messages.tail()

,mid,from_clean,user,type,subject,body,date,bcc,uuid,eid
473744,117670,periwinklefoundation.org,bailey-s,received,The Periwinkle Foundation,Please check this list to see if this is how ...,"Tue, 29 Jan 2002 14:57:05",None,146,428
473745,117671,periwinklefoundation.org,bailey-s,received,Board Retreat,Attached is an announcement about The Periwin...,"Thu, 14 Mar 2002 14:36:45",None,146,428
473746,118523,bw-legal.com,bailey-s,received,I am not a jerk Christmas gifts,Susan I just received a return of my mailing ...,"Mon, 14 Jan 2002 08:30:37",None,146,432
473747,118524,bw-legal.com,bailey-s,received,no subject,Thanks And Love too Bailey Susan AM Dan Just ...,"Mon, 26 Nov 2001 11:21:07",None,146,432
473748,118525,bw-legal.com,bailey-s,received,Discussion with John Miller re Willow Farms,Background As you may remember in September J...,"Wed, 30 Jan 2002 12:09:26",None,146,432


In [37]:
dates=df_messages['date'].tolist()
times=[i[-9:-2] for i in dates]

In [38]:
df_messages['times']=times

In [39]:
df_messages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 473749 entries, 0 to 473748
Data columns (total 11 columns):
mid           473749 non-null int64
from_clean    473749 non-null object
user          473749 non-null object
type          473749 non-null object
subject       473749 non-null object
body          473749 non-null object
date          473749 non-null object
bcc           473749 non-null object
uuid          473749 non-null int64
eid           473749 non-null int64
times         473749 non-null object
dtypes: int64(3), object(8)
memory usage: 63.4+ MB


In [40]:
#infer date
df_messages['date'] = pd.to_datetime(df_messages['date'],infer_datetime_format = True)

In [41]:
df_messages.body[1337]

' Current Gas Desk List Hope this helps This is the current gas desk list that I have in my personal address book Call me with any questions Erika GROUP East Desk Basics Group name East Desk Group type Multi purpose Description Members Matthew B Fleming HOU EES James R Barker HOU EES Barend VanderHorst HOU EES Jay Blaine HOU EES Paul Tate HOU EES Alain Diza HOU EES Rhonda Smith HOU EES Christina Bangle HOU EES Sherry Pendegraft HOU EES Marde L Driscoll HOU EES Daniel Salinas HOU EES Sharon Hausinger HOU EES Joshua Bray HOU EES James Wiltfong HOU EES Owners Erika Dupre HOU EES Administrators Erika Dupre HOU EES Foreign directory sync allowed Yes GROUP West Desk Basics Group name West Desk Group type Multi purpose Description Members Jesus Guerra HOU EES Monica Roberts HOU EES Laura R Arnold HOU EES Amanda Boettcher HOU EES C Kyle Griffin HOU EES Jess Hewitt HOU EES Artemio Muniz HOU EES Eugene Zeitz HOU EES Brandon Whittaker HOU EES Roland Aguilar HOU EES Ruby Robinson HOU EES Roger Rey

In [42]:
df_messages['unique']= df_messages['subject'].map(str)+ '-' +df_messages['from_clean'].map(str) + '-' + df_messages['date'].map(str) + '-' + df_messages['times'].map(str)

In [43]:
df_messages.unique.unique()[138168]

' Master Agreement Structural Issues Working Group Meeting Wed nesd ay March at a.m. NY time p.m. London time -isda-eur.org-2001-03-20 09:08:00-09:08:0'

In [44]:
df_messages[df_messages.unique=='Memo on Mitigation Measures -sarah.novosel@enron.com-2000-08-04 02:42:00-02:42:0']

,mid,from_clean,user,type,subject,body,date,bcc,uuid,eid,times,unique
4024,4011,sarah.novosel@enron.com,kean-s,received,Memo on Mitigation Measures,As we discussed yesterday attached is a memor...,2000-08-04 02:42:00,None,4,13,02:42:0,Memo on Mitigation Measures -sarah.novosel@enr...
4032,4019,sarah.novosel@enron.com,kean-s,received,Memo on Mitigation Measures,As we discussed yesterday attached is a memor...,2000-08-04 02:42:00,None,4,13,02:42:0,Memo on Mitigation Measures -sarah.novosel@enr...
4095,4082,sarah.novosel@enron.com,kean-s,received,Memo on Mitigation Measures,As we discussed yesterday attached is a memor...,2000-08-04 02:42:00,None,4,13,02:42:0,Memo on Mitigation Measures -sarah.novosel@enr...
4156,4143,sarah.novosel@enron.com,kean-s,received,Memo on Mitigation Measures,As we discussed yesterday attached is a memor...,2000-08-04 02:42:00,None,4,13,02:42:0,Memo on Mitigation Measures -sarah.novosel@enr...
4205,4192,sarah.novosel@enron.com,kean-s,received,Memo on Mitigation Measures,As we discussed yesterday attached is a memor...,2000-08-04 02:42:00,None,4,13,02:42:0,Memo on Mitigation Measures -sarah.novosel@enr...
4247,4234,sarah.novosel@enron.com,kean-s,received,Memo on Mitigation Measures,As we discussed yesterday attached is a memor...,2000-08-04 02:42:00,None,4,13,02:42:0,Memo on Mitigation Measures -sarah.novosel@enr...


In [45]:
mid, unique = pd.factorize(df_messages.unique.tolist(),sort =True)

In [46]:
df_messages['mid']=mid

In [47]:
df_messages[df_messages.unique==' Master Agreement Structural Issues Working Group Meeting Wed nesd ay March at a.m. NY time p.m. London time -isda-eur.org-2001-03-20 09:08:00-09:08:0']

,mid,from_clean,user,type,subject,body,date,bcc,uuid,eid,times,unique
302812,46639,isda-eur.org,taylor-m,received,Master Agreement Structural Issues Working Gr...,Please find attached a slightly revised by Jo...,2001-03-20 09:08:00,None,11,6860,09:08:0,Master Agreement Structural Issues Working Gr...
302819,46639,isda-eur.org,taylor-m,received,Master Agreement Structural Issues Working Gr...,Please find attached a slightly revised by Jo...,2001-03-20 09:08:00,None,11,6860,09:08:0,Master Agreement Structural Issues Working Gr...


In [48]:
df_messages.drop(columns = ['times','unique'],inplace=True)

In [49]:
df_messages.to_csv('D:/Capstone/dataset/database/messages.csv')